In [ ]:
# Importando as Bibliotecas
import pandas as pd
import requests

from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
#Informações para fingir ser um navegador
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

#A url que você quer acesssar
url_link1 = "https://www.soccerstats.com/matches.asp?matchday=1&listing=1"
url_link2 = "https://www.soccerstats.com/matches.asp?matchday=1&listing=2"

#juntamos tudo com a requests
r1 = requests.get(url_link1, headers=header)
r2 = requests.get(url_link2, headers=header)

#E finalmente usamos a função read_html do pandas
df1 = pd.read_html(r1.text)
df2 = pd.read_html(r2.text)

In [ ]:
df1[6]

In [ ]:
df2[6]

In [ ]:
# GP: number of matches played
# W%: percentage of matches won
# FTS: % Failed to score
# CS: % Clean sheet
# BTS: % of matches where both teams scored 
# TG: Average total goals scored + conceded per match
# GF: goals scored per match 
# GA: goals conceded per match
# 1.5+: % of matches over 1.5 goals (scored + conceded)
# 2.5+: % of matches over 2.5 goals (scored + conceded) 
# 3.5+: % of matches over 3.5 goals (scored + conceded)
# PPG: Points per game

In [ ]:
jogos_today1 = df1[6]
jogos_today1 = jogos_today1[['Country','2.5+','1.5+','GA','GF','TG','PPG','Unnamed: 9','Unnamed: 10','Unnamed: 11','PPG.1','TG.1','GF.1','GA.1','1.5+.1','2.5+.1']]
jogos_today1.columns = ['País','Over25_H','Over15_H','GolsSofridos_H','GolsMarcados_H','MediaGols_H','PPG_H','Home','Hora','Away','PPG_A','MediaGols_A','GolsMarcados_A','GolsSofridos_A','Over15_A','Over25_A']

In [ ]:
jogos_today2 = df2[6]
jogos_today2 = jogos_today2[['BTS','W%','BTS.1','W%.1']]
jogos_today2.columns = ['BTTS_H','%Vitorias_H','BTTS_A','%Vitorias_A']

In [ ]:
jogos_today = pd.concat([jogos_today1,jogos_today2],axis=1)
jogos_today = jogos_today[['País','Hora','Home','Away','%Vitorias_H','%Vitorias_A','Over15_H','Over25_H','Over15_A','Over25_A','BTTS_H','BTTS_A','GolsMarcados_H','GolsSofridos_H','GolsMarcados_A','GolsSofridos_A','MediaGols_H','MediaGols_A','PPG_H','PPG_A']]

In [ ]:
jogos_today = jogos_today.sort_values('Hora')
jogos_today['Hora'] = pd.to_datetime(jogos_today['Hora']) - pd.DateOffset(hours=4)
jogos_today['Hora'] = pd.to_datetime(jogos_today['Hora'], format='%H:%M').dt.time
jogos_today = jogos_today.dropna()
# Resetando o Index
jogos_today.reset_index(inplace=True, drop=True)
jogos_today.index = jogos_today.index.set_names(['Nº'])
jogos_today = jogos_today.rename(index=lambda x: x + 1)
# Exportando para o Excel
jogos_today.to_excel("Jogos de Hoje.xlsx")

In [ ]:
df = jogos_today
df

In [ ]:
flt = (df.GolsMarcados_H > 1) & (df.GolsSofridos_H > 1) & (df.GolsMarcados_A > 1) & (df.GolsSofridos_A > 1) & (df.MediaGols_H > 1) & (df.MediaGols_A > 1)
df1 = df[flt]
df1

In [ ]:
df.info()